In [1]:
import gym
from gym import spaces
from gym.utils import seeding
import itertools
import logging
import numpy as np
from collections import deque
import itertools

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.optimizers import Adam
from keras import layers

In [4]:
#Importing the Game Board
from Game_Board import*
max_tile=[]

In [5]:
#Converting observations in range (0,1) using log(n)/log(max) so that gradients don't vanish
def process_log(observation):
        observation = np.reshape(observation, (4, 4))
        observation_temp = np.where(observation <= 0, 1, observation) 
        processed_observation = np.log2(observation_temp)/np.log2(65536)
        return processed_observation.reshape(1,4,4)

In [6]:
def get_grids_next_step(grid):
        #Returns the next 4 states s' from the current state s
        
        grids_list = []
        
        for movement in range(4):
            grid_before = grid.copy()
            env1=Game2048Env()
            env1.set_board(grid_before)
            try:
                _= env1.move(movement) 
            except:
                pass
            grid_after =env1.get_board()
            env1.get_board()
            grids_list.append(grid_after)
            
        return grids_list

In [7]:
class DQN:
    def __init__(self,env):
        #Defining the hyperparameters for the model
        self.env=env.get_board()
        #The replay memory will be stored in a Deque
        self.memory=deque(maxlen=2000)
        self.gamma=0.90
        self.epsilon=1.0
        self.epsilon_min=0.01
        self.epsilon_decay=0.995
        self.learning_rate=0.005
        self.tau=0.125
        #We use 2 models to prevent Bootstrapping
        self.model=self.create_model()
        self.target_model=self.create_model()
        
        
    def create_model(self):
        model=Sequential()
        state_shape=4
                
        model.add(Flatten(input_shape=(4,4)))
        model.add(Dense(units=1024,activation="relu"))
        model.add(Dense(units=512,activation="relu"))
        model.add(Dense(units=256,activation="relu"))
        model.add(Dense(units=4))
        model.compile(loss="mean_squared_error",optimizer=Adam(lr=self.learning_rate))
        print(model.summary())
        return model
                  
    def act(self,state):
        #Epsilon value decays as model gains experience
        self.epsilon*=self.epsilon_decay
        self.epsilon=max(self.epsilon_min,self.epsilon)
        if np.random.random()<self.epsilon:
                  return np.random.randint(0,4)
        else:
            #Getting the 4 future states
            allstates=get_grids_next_step(state)
            
            res=[]
            for i in range(len(allstates)):
                if (allstates[i]==state).all():
                    res.append(0)
                else:
                    processed_state=process_log(allstates[i])
                    #max from the 4 future Q_Values is appended in res
                    res.append(np.max(self.model.predict(processed_state)))
            
            a=self.model.predict(process_log(state))
            #Final Q_Values are the sum of Q_Values of current state andfuture states
            final=np.add(a,res)
            
            return np.argmax(final)
    
    def remember(self, state, action, reward, new_state, done):
        #Replay Memory stores tuple(S, A, R, S')
        self.memory.append([state, action, reward, new_state, done])

    def replay(self):
        batch_size=32
        if len(self.memory)<batch_size:
            return
        samples=random.sample(self.memory,batch_size)
        for sample in samples:
            
            state,action,reward,new_state,done=sample
            
            target=self.target_model.predict(process_log(state))
            
            
            if done:
                target[0][action]=reward
            else:
                #Bellman Equation for update
                Q_future=max(self.target_model.predict(process_log(new_state))[0])
                
                #The move which was selected, its Q_Value gets updated
                target[0][action]=reward+Q_future*self.gamma
            self.model.fit((process_log(state)),target,epochs=1,verbose=0)
                  
                  
    def target_train(self):
        weights=self.model.get_weights()
        target_weights=self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i]=weights[i]*self.tau+target_weights[i]*(1-self.tau)
        self.target_model.set_weights(target_weights)
                  
                  
    def save_model(self,fn):
        self.model.save(fn)
                  
                  


In [9]:

def main():
    env=Game2048Env()
    gamma=0.9
    epsilon=0.95
    
    trials=2000
    trial_len=500
    dqn_agent=DQN(env=env)
    steps=[]
    
    
    
    for trial in range(trials):
        env.reset()
        cur_state=env.get_board()
        
        cur_state=cur_state.reshape(4,4)
        stepno=0
        
        for step in range(trial_len):
            stepno+=1
            
            action=dqn_agent.act(cur_state)
            
            _,reward,done,_=env.step(action)
            new_state=env.get_board()
            
            dqn_agent.remember(cur_state,action,reward,new_state,done)
            if step%10==0:
                dqn_agent.replay()
                dqn_agent.target_train()
            
            cur_agent=new_state
            
            if done:
                
                break
        
        if stepno<500:
            
            if env.highest()==2048:
                print("Completed in --",trial)
                print(env.get_board())
                max_tile.append(env.highest())

                dqn_agent.save_model("success.model")

            
            else:
                print(f"Trial number {trial} Failed to complete in 500 steps")
                print(env.get_board())
                max_tile.append(env.highest())
                if env.highest()>=512:
                    dqn_agent.save_model("trial-{}.model".format(trial))
                    
        else:
            print(f"Failed to complete in 500 steps")
            print(env.get_board())
            max_tile.append(env.highest())

            
if __name__ == "__main__":
    main()  

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 16)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              17408     
_________________________________________________________________
dense_10 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_11 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_12 (Dense)             (None, 4)                 1028      
Total params: 674,564
Trainable params: 674,564
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_4"
_________________________________________________________________
Layer (type)         

Failed to complete in trial number 50 in 500 steps
[[  2   8   2   4]
 [  4  16   8  32]
 [  2  32   4  64]
 [  4  16  64 128]]
Failed to complete in trial number 51 in 500 steps
[[  2   4   2  16]
 [  4   8  16  64]
 [  8  16  64 128]
 [  2   4   8 256]]
Failed to complete in trial number 52 in 500 steps
[[  2   4  16  32]
 [  4  16  64   2]
 [ 16   8 128 256]
 [  2   4   8  32]]
Failed to complete in trial number 53 in 500 steps
[[  4   2   4  16]
 [  2   8   2  64]
 [  4  32   4 128]
 [  2  16  64 256]]
Failed to complete in trial number 54 in 500 steps
[[  2   4   8  16]
 [  4   8  32  64]
 [  8  16 128   2]
 [  2  32  16 128]]
Failed to complete in trial number 55 in 500 steps
[[  4   2   4  32]
 [  2   4  16   4]
 [  4   8   2 128]
 [  2  32  64 256]]
Failed to complete in trial number 56 in 500 steps
[[  2   4   8   2]
 [  4   8   2  32]
 [  8  32  16 256]
 [ 16   4  64   4]]
Failed to complete in trial number 57 in 500 steps
[[ 2  4  8 16]
 [ 4  8 16 32]
 [ 8 16 32 64]
 [ 2 32 

Failed to complete in trial number 115 in 500 steps
[[ 2  4  2  8]
 [ 4 16  4 32]
 [16  2 16 64]
 [ 2  8 64  8]]
Failed to complete in trial number 116 in 500 steps
[[  4   2   4   2]
 [  2   4   8   4]
 [  4  16   2  16]
 [  2  32   8 256]]
Failed to complete in trial number 117 in 500 steps
[[  2   4   8  16]
 [  4   8  32  64]
 [  8  16  64 128]
 [ 16  64 128   2]]
Failed to complete in trial number 118 in 500 steps
[[  4   2   4  16]
 [  2   4   8  32]
 [  4  16  32   2]
 [  2   8  16 128]]
Failed to complete in trial number 119 in 500 steps
[[  2   4  16  32]
 [  4   8  32   2]
 [  8  16  64   8]
 [  2  32 128 256]]
Failed to complete in trial number 120 in 500 steps
[[  2   4   8  16]
 [  4   8  16   2]
 [  2  32  64   8]
 [  8   2  16 256]]
Failed to complete in trial number 121 in 500 steps
[[  4   2   4  16]
 [  8  16  32   2]
 [  4  32   4   8]
 [  2  64   2 256]]
Failed to complete in trial number 122 in 500 steps
[[  4   2   4  16]
 [  2   4  16  32]
 [  4   8   2  16]
 [  

Failed to complete in trial number 180 in 500 steps
[[ 4  2  4 32]
 [ 2  8 16 64]
 [ 8 16  2  4]
 [ 2  4  8 64]]
Failed to complete in trial number 181 in 500 steps
[[  8   2   4   8]
 [  2   4  16  64]
 [  4   8  64 128]
 [  2   4 128 256]]
Failed to complete in trial number 182 in 500 steps
[[  2   4   8  16]
 [  4   8  16  32]
 [  8  16  32 128]
 [  2   4   2   8]]
Failed to complete in trial number 183 in 500 steps
[[ 2  4  8 16]
 [ 4  8 16 32]
 [ 8 16 32  8]
 [16 32  2 32]]
Failed to complete in trial number 184 in 500 steps
[[  4   2   8   2]
 [  2   8  16  32]
 [  8  16   2  64]
 [  2   8  64 128]]
Failed to complete in trial number 185 in 500 steps
[[  4   2   4   8]
 [  8  32   8  16]
 [  2   8  16  64]
 [  8  32   4 128]]
Failed to complete in trial number 186 in 500 steps
[[  8   2   4  16]
 [  2   4  32   2]
 [  4  32  64 128]
 [  2  64   2 256]]
Failed to complete in trial number 187 in 500 steps
[[  4   2   4   8]
 [  2   4   8  64]
 [  8  16  64 128]
 [  2  32 128   2]]


Failed to complete in trial number 246 in 500 steps
[[  2   4   8  16]
 [  4   8  16  32]
 [  8  16  32  64]
 [  2   8  64 128]]
Failed to complete in trial number 247 in 500 steps
[[ 4  2  4 16]
 [ 2  4 16 32]
 [32  8 32  4]
 [ 2 64  2 64]]
Failed to complete in trial number 248 in 500 steps
[[  2   4  16  32]
 [  4   8  32  64]
 [  8  16  64 256]
 [  2   4  16  32]]
Failed to complete in trial number 249 in 500 steps
[[  2   4   2   4]
 [  4  16   4  16]
 [  8   4  16  64]
 [ 16   8  32 256]]
Failed to complete in trial number 250 in 500 steps
[[  2   4   8  16]
 [  4   8  16  64]
 [  8  32  64 128]
 [ 16   2   8  16]]
Failed to complete in trial number 251 in 500 steps
[[  2   4   8  16]
 [  4   8  16   4]
 [ 16  32   4  32]
 [  2   4  64 128]]
Failed to complete in trial number 252 in 500 steps
[[ 4  2  8  2]
 [16  8  2  8]
 [ 4  2  4 16]
 [ 2  8 16  2]]
Failed to complete in trial number 253 in 500 steps
[[  2   4   8  16]
 [  4   8  32  64]
 [  8   2  64 128]
 [  2  64 128 512]]


...

Failed to complete in trial number 509 in 500 steps
[[  4   2   8   2]
 [  2   8  16  32]
 [  8  16  32  64]
 [  2  32   8 512]]
Failed to complete in trial number 510 in 500 steps
[[  2   4   8   4]
 [  4   8  32  64]
 [ 16  32  64   8]
 [  2  64 128 256]]
Failed to complete in trial number 511 in 500 steps
[[  2   4   8   2]
 [  4   8  16  32]
 [  8  32  64   2]
 [ 32   2 256  16]]
Failed to complete in trial number 512 in 500 steps
[[ 4  8 16  4]
 [ 2  4  2 16]
 [ 4  2  4 32]
 [ 2  8 16 64]]
Failed to complete in trial number 513 in 500 steps
[[  4   2   8  16]
 [  2   4  32   4]
 [  4  32  64   8]
 [  8  64 128 256]]
Failed to complete in trial number 514 in 500 steps
[[  4  16   2   4]
 [  2   4  32  64]
 [  4   8   2   4]
 [  8  16  32 256]]
Failed to complete in trial number 515 in 500 steps
[[ 2  4 16  2]
 [ 4  8 32  8]
 [ 8  2  8 16]
 [ 2  8 16 32]]
Failed to complete in trial number 516 in 500 steps
[[  2   4   8   4]
 [  4  16  32  16]
 [  8  32  64 128]
 [ 32  64 256   4]]


Failed to complete in trial number 575 in 500 steps
[[  2   4   8   2]
 [  4   8  32   4]
 [  8  32 128   2]
 [  4  16   2   4]]
Failed to complete in trial number 576 in 500 steps
[[  2   4   8   4]
 [  4   8  16  32]
 [  8  16   2 128]
 [ 16   4  64   8]]
Failed to complete in trial number 577 in 500 steps
[[  2   4   8   2]
 [  4   8  16   4]
 [  8  16   2 128]
 [ 16   2  64   2]]
Failed to complete in trial number 578 in 500 steps
[[  4   2   4   8]
 [  2   4   8  32]
 [  8  16  32  64]
 [  2  64   2 256]]
Failed to complete in trial number 579 in 500 steps
[[  4   2   4  16]
 [  8  64  16  32]
 [  2  16  32  64]
 [ 16   4   8 128]]
Failed to complete in trial number 580 in 500 steps
[[  4   2   4  16]
 [  2   8  16  32]
 [  8  16  32  64]
 [  2  32 128 256]]
Failed to complete in trial number 581 in 500 steps
[[  2   4   8   2]
 [  4   8   4  32]
 [  8  16  32   4]
 [  2  64 128   8]]
Failed to complete in trial number 582 in 500 steps
[[  2   4   8  32]
 [  4  32   4  16]
 [ 16  

Failed to complete in trial number 640 in 500 steps
[[  2   8   2  16]
 [ 16   4   8  32]
 [  4   8  32   2]
 [  2  16   2 128]]
Failed to complete in trial number 641 in 500 steps
[[  4   2   8   4]
 [  2   8  16  32]
 [  4  16  64 128]
 [ 16   2   8  16]]
Failed to complete in trial number 642 in 500 steps
[[  2   4  16   4]
 [  4  16  32   8]
 [ 16  32  64  16]
 [ 32   2   8 512]]
Failed to complete in trial number 643 in 500 steps
[[  2   4  32   2]
 [  4   8  16   4]
 [  8  16  32  16]
 [ 16  32 128   2]]
Failed to complete in trial number 644 in 500 steps
[[  2   4   8  32]
 [  4  16  32   2]
 [  2   8 128   4]
 [  8  16  32  64]]
Failed to complete in trial number 645 in 500 steps
[[  2   4   8   4]
 [  4   2  16  32]
 [  2  16  32   4]
 [ 16  32  64 512]]
Failed to complete in trial number 646 in 500 steps
[[ 2  8  2  4]
 [ 8  2 16 32]
 [ 2  4 32 64]
 [ 8 16 64  2]]
Failed to complete in trial number 647 in 500 steps
[[  4   8   4   2]
 [  2  16   2  16]
 [  4   8  32  64]
 [ 1

Failed to complete in trial number 705 in 500 steps
[[  4   2   4  16]
 [  2   4   8  32]
 [  4   8  16  64]
 [ 16  32  64 256]]
Failed to complete in trial number 706 in 500 steps
[[  2   4   8  16]
 [  4   8  16  32]
 [  8  16  32 256]
 [  4  32  64   8]]
Failed to complete in trial number 707 in 500 steps
[[  2   4   2  16]
 [  4   2  16   4]
 [  2   8  32 128]
 [  4  32  64 256]]
Failed to complete in trial number 708 in 500 steps
[[  4   8  16  32]
 [  2  16  32 128]
 [  4  64 128   2]
 [  2   8  32 256]]
Failed to complete in trial number 709 in 500 steps
[[  2   4   8  16]
 [  4   8  16  32]
 [ 16   2  64 128]
 [  4   8 128   2]]
Failed to complete in trial number 710 in 500 steps
[[ 2  4  8 16]
 [ 4  2 16 32]
 [ 2  8 32 64]
 [ 8 16 64  4]]
Failed to complete in trial number 711 in 500 steps
[[  2   4   8  16]
 [  4   8  32  64]
 [  8  16  64 128]
 [  2   4   8  32]]
Failed to complete in trial number 712 in 500 steps
[[ 2  8  2 16]
 [ 8  4  8 32]
 [32 16  2 16]
 [ 2  4 32 64]]


KeyboardInterrupt: 